# Project 2

# Exploring the 311 service request

We continued to use the 311 service request for analyzing. This time I chose to add 'Rat‑Related 311 Service Requests'to the project which specifically focuses on rat-related complaints via 311. Let's see the relationship between them.

# Step 1
install the plotly for visualization and get prepared with the datasets. 

In [1]:
%pip install plotly
from IPython.display import HTML


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import plotly.express as px

url_all = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$limit=50000"
url_rat = "https://data.cityofnewyork.us/resource/cbqt-vkjz.csv?$limit=50000"

df_all = pd.read_csv(url_all, parse_dates=["created_date"])
df_rat = pd.read_csv(url_rat, parse_dates=["created_date"])

/var/folders/bj/gpmqq0dx5ld8w1w42kl86m2h0000gn/T/ipykernel_97022/3139748308.py:7: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(url_all, parse_dates=["created_date"])


# Step 2 
Cleaning the data and aggregate by day + borough

In [3]:
# create a 'day' column (date only)
df_all["day"] = df_all["created_date"].dt.to_period("D")
df_rat["day"] = df_rat["created_date"].dt.to_period("D")

print(df_all)
print(df_rat)

# aggregate by day + borough
agg_all = df_all.groupby(["borough", "day"]).size().reset_index(name="total_requests")
agg_rat = df_rat.groupby(["borough", "day"]).size().reset_index(name="rat_requests")

       unique_key        created_date              closed_date agency  \
0        66915771 2025-11-22 02:06:25                      NaN    TLC   
1        66914322 2025-11-22 02:06:25                      NaN   NYPD   
2        66911489 2025-11-22 02:05:59                      NaN   NYPD   
3        66908585 2025-11-22 02:05:13                      NaN   NYPD   
4        66908621 2025-11-22 02:04:27                      NaN   NYPD   
...           ...                 ...                      ...    ...   
49995    66861067 2025-11-17 10:55:17                      NaN    HPD   
49996    66863326 2025-11-17 10:55:06  2025-11-17T21:26:45.000   NYPD   
49997    66862069 2025-11-17 10:55:04  2025-11-17T12:17:05.000   NYPD   
49998    66862077 2025-11-17 10:55:03  2025-11-17T11:40:00.000   NYPD   
49999    66869357 2025-11-17 10:55:00  2025-11-17T11:40:00.000    DOT   

                                             agency_name  \
0                          Taxi and Limousine Commission   
1  

# Step 3
Merge and compute ratio, then convert Period to real datetime for Plotly.

In [4]:

df_merge = pd.merge(agg_all, agg_rat, on=["borough", "day"], how="left")
df_merge["rat_requests"] = df_merge["rat_requests"].fillna(0)
df_merge["rat_ratio"] = df_merge["rat_requests"] / df_merge["total_requests"]

df_merge["day"] = df_merge["day"].dt.to_timestamp()

In [5]:
df_all["created_date"].min(), df_all["created_date"].max(), df_rat["created_date"].min(), df_rat["created_date"].max()

(Timestamp('2025-11-17 10:55:00'),
 Timestamp('2025-11-22 02:06:25'),
 Timestamp('2025-09-13 17:41:19'),
 Timestamp('2025-11-22 01:54:00'))

# Step 4
Show the final result of the visualization.

In [6]:
%pip install kaleido


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
fig = px.line(
    df_merge,
    x="day",
    y="rat_ratio",
    color="borough",
    title="Rat Complaints as % of All 311 Requests (by Borough, Over Days)",
    labels={
        "day": "Day",
        "rat_ratio": "Rat Requests / Total 311 Requests",
        "borough": "Borough"
    }
)
fig.write_image("rat_ratio.png")  
# Wherever you have fig.show() replace it with this code:
HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))

# Conclusion
The rat-related complaints consistently make up a measurable share of 311 activity, with Manhattan and Brooklyn showing the highest ratios.
This implies that rodent issues are strongly associated with dense urban areas, older building stock, and high commercial activity—patterns confirmed through the relationship between these two datasets.